# Librerías

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
from limpieza import *

In [3]:
#!pip install ipython-autotime
%load_ext autotime

In [4]:
data = 'D:/Descargas/datos_gps_curso_ciencia_de_datos/gps_filtered.csv'
processed_path = 'D:/Descargas/datos_gps_curso_ciencia_de_datos/gps_processed.csv'
train_path = 'D:/Descargas/datos_gps_curso_ciencia_de_datos/train.csv'
val_path = 'D:/Descargas/datos_gps_curso_ciencia_de_datos/val.csv'
test_path = 'D:/Descargas/datos_gps_curso_ciencia_de_datos/test.csv'
mean_path = 'mean.csv'
std_path = 'std.csv'
test_days = [28, 29, 30, 31]
train_val_perc = 0.2
thresh = 50         # Minima cantidad de pulsos en una expedicion para ser considerada
delta_dist = 100    # Distancia en metros minima entre dos pulsos para ser pareados
n = 10               # Multiplicador de cantidad de pulsos pares maximos, n igual a 1 los pares de pulsos son igual a la cantida de pulsos
jump = 3            # Cantidad de indices que se salta al realizar pares de pulsos

time: 999 µs


# Lectura base de datos

In [5]:
gps = pd.read_csv(data, sep=",")
gps.head()

,Unnamed: 0,DistanciaInicio,DistanciaRuta,GPS_time,Ind1,Ind2,Ind3,Latitud,Longitud,Patente,Servicio,Unnamed: 0.1,Unnamed: 0.1.1
0,0,0,147,2019-08-01 00:00:28,0,5,0,-33.543559,-70.789180,BDXR-15,T506 00I,0,12057.0
1,1,0,140,2019-08-01 00:00:15,0,5,0,-33.543655,-70.788752,BFKB-85,T506 00I,1,12058.0
2,2,0,140,2019-08-01 00:00:45,0,5,0,-33.543655,-70.788752,BFKB-85,T506 00I,2,12059.0
3,3,0,140,2019-08-01 00:01:15,0,5,0,-33.543655,-70.788752,BFKB-85,T506 00I,3,12060.0
4,4,0,139,2019-08-01 00:01:45,5,5,0,-33.543637,-70.788783,BFKB-85,T506 00I,4,12061.0


time: 32.2 s


# Limpieza de base de datos

In [7]:
gps = limpieza(gps)

time: 15.9 s


In [8]:
gps.head()

,Unnamed: 0,DistanciaInicio,DistanciaRuta,GPS_time,Ind1,Ind2,Ind3,Latitud,Longitud,Patente,Servicio,Unnamed: 0.1,Unnamed: 0.1.1,dia_habil,dia_Monday,dia_Saturday,dia_Sunday,dia_Thursday,dia_Tuesday,dia_Wednesday
0,0,0,147,2019-08-01 00:00:28,0,5,0,-33.543559,-70.789180,BDXR-15,T506 00I,0,12057.0,1,0,0,0,1,0,0
1,1,0,140,2019-08-01 00:00:15,0,5,0,-33.543655,-70.788752,BFKB-85,T506 00I,1,12058.0,1,0,0,0,1,0,0
2,2,0,140,2019-08-01 00:00:45,0,5,0,-33.543655,-70.788752,BFKB-85,T506 00I,2,12059.0,1,0,0,0,1,0,0
3,3,0,140,2019-08-01 00:01:15,0,5,0,-33.543655,-70.788752,BFKB-85,T506 00I,3,12060.0,1,0,0,0,1,0,0
4,4,0,139,2019-08-01 00:01:45,5,5,0,-33.543637,-70.788783,BFKB-85,T506 00I,4,12061.0,1,0,0,0,1,0,0


time: 26 ms


# Separación de expediciones

In [8]:
gps["GPS_time"] = pd.to_datetime(gps["GPS_time"])
gps = gps.sort_values(by=['GPS_time'])

time: 24.3 s


In [9]:
%load_ext Cython

time: 1.33 s


In [10]:
%%cython
import cython
import numpy as np
cimport numpy as np

cpdef float [:] expeditions(float [:, :] gps, float delta, float epsilon):
    cdef np.ndarray[np.float32_t, ndim=1] output = np.zeros((gps.shape[0]), dtype=np.float32)
    cdef int i
    cdef int n_exp = 0

    for i in range(gps.shape[0] - 1):
        output[i] = n_exp
        if np.abs(gps[i+1, 0] - gps[i, 0]) + np.abs(gps[i+1, 1] - gps[i, 1]) <= epsilon:
            output[i] = -1
        elif np.abs(gps[i+1, 0] - gps[i, 0]) > delta:
            n_exp += 1
    output[-1] = n_exp
    
    return output

time: 22 ms


In [11]:
def get_expeditions(gps, delta, epsilon=-1):
    aux = gps.sort_values(by=['GPS_time'])[["DistanciaInicio", "DistanciaRuta"]].to_numpy(dtype=np.float32)
    expedition = np.asarray(expeditions(aux, delta, epsilon))
    return expedition

time: 1 ms


In [12]:
def add_expedition(gps, alpha=0.5, delta=0):
    servicios = gps["Servicio"].unique()
    total = len(servicios)
    for idy, servicio in enumerate(servicios):
        print('Servicio {}/{}'.format(idy+1, total))
        gps_servicio = gps[gps["Servicio"] == servicio]
        patentes = gps_servicio["Patente"].unique()
        total_pat = len(patentes)
        for idx, patente in enumerate(patentes):
            print('\rPatente {}/{}'.format(idx+1, total_pat), end='')
            gps_patente = gps_servicio[gps_servicio["Patente"] == patente].sort_values(by=['GPS_time'])
            exp_max = np.max(gps_patente["DistanciaInicio"])
            gps_patente["Expedicion"] = get_expeditions(gps_patente, alpha*exp_max, delta).astype(int)
            gps_patente = gps_patente[gps_patente["Expedicion"] >= 0]
            gps_patente["exp_id"] = gps_patente["Patente"] + "_" + gps_patente["Expedicion"].astype(str)
            gps_patente = gps_patente.drop(columns=["Patente", "Expedicion"])
            if idx == 0 and idy == 0:
                nuevo_gps = gps_patente.to_numpy()
                headers = list(gps_patente.columns)
            else:
                nuevo_gps = np.concatenate((nuevo_gps, gps_patente.to_numpy()), axis=0)
        print('\n')
    return pd.DataFrame(nuevo_gps, columns=headers)

time: 5 ms


In [13]:
gps = add_expedition(gps)
gps

Servicio 1/19
Patente 273/273

Servicio 2/19
Patente 183/183

Servicio 3/19
Patente 240/240

Servicio 4/19
Patente 227/227

Servicio 5/19
Patente 378/378

Servicio 6/19
Patente 249/249

Servicio 7/19
Patente 241/241

Servicio 8/19
Patente 285/285

Servicio 9/19
Patente 201/201

Servicio 10/19
Patente 273/273

Servicio 11/19
Patente 224/224

Servicio 12/19
Patente 217/217

Servicio 13/19
Patente 303/303

Servicio 14/19
Patente 293/293

Servicio 15/19
Patente 350/350

Servicio 16/19
Patente 303/303

Servicio 17/19
Patente 204/204

Servicio 18/19
Patente 231/231

Servicio 19/19
Patente 237/237



,Unnamed: 0,DistanciaInicio,DistanciaRuta,GPS_time,Ind1,Ind2,Ind3,Latitud,Longitud,Servicio,Unnamed: 0.1,Unnamed: 0.1.1,dias,dia_habil,exp_id
0,2766176,6764,5,2016-02-16 08:51:07,0,2,0,-33.5555,-70.5862,T210 00R,2766176,5.19394e+06,Tuesday,1,WA-9717_0
1,2766667,666,2180,2019-07-02 12:08:23,0,2,0,-33.6055,-70.552,T210 00R,2766667,5.39835e+06,Tuesday,1,ZN-6314_0
2,74081,739,2202,2019-08-21 17:44:21,0,2,0,-33.6048,-70.5519,T210 00R,10183660,NaN,Wednesday,1,ZN-6314_0
3,785830,680,2176,2019-08-22 13:25:48,0,2,0,-33.6054,-70.552,T210 00R,6912564,NaN,Thursday,1,ZN-6314_0
4,789721,650,2173,2019-08-22 16:22:01,0,2,0,-33.6056,-70.552,T210 00R,8764786,NaN,Thursday,1,ZN-6314_0
5,790040,660,2186,2019-08-22 16:40:50,0,2,0,-33.6055,-70.5519,T210 00R,8959905,NaN,Thursday,1,ZN-6314_0
6,793257,679,2172,2019-08-22 18:58:53,0,2,0,-33.6054,-70.5521,T210 00R,10558545,NaN,Thursday,1,ZN-6314_0
7,1487218,659,2159,2019-08-23 18:39:10,0,2,0,-33.6056,-70.5522,T210 00R,9915284,NaN,Friday,1,ZN-6314_0
8,1494661,785,2202,2019-08-23 23:11:13,0,2,0,-33.6044,-70.5519,T210 00R,12862703,NaN,Friday,1,ZN-6314_0
9,2831693,652,2176,2019-08-26 19:32:16,0,2,0,-33.6056,-70.552,T210 00R,10618178,NaN,Monday,1,ZN-6314_0


time: 2h 56min 40s


In [24]:
gps.to_csv('D:/Descargas/datos_gps_curso_ciencia_de_datos/checkpoint.csv', sep=',', index=False, mode='a+')

time: 2min 16s


# Pares de pulsos

In [5]:
gps = pd.read_csv('D:/Descargas/datos_gps_curso_ciencia_de_datos/checkpoint_horario.csv', sep=',')
gps["GPS_time"] = pd.to_datetime(gps["GPS_time"])
gps.head()

,Unnamed: 0,DistanciaInicio,DistanciaRuta,GPS_time,Ind1,Ind2,Ind3,Latitud,Longitud,Servicio,...,Unnamed: 0.1.1,dia_habil,dia_Monday,dia_Saturday,dia_Sunday,dia_Thursday,dia_Tuesday,dia_Wednesday,exp_id,horario
0,182891,29940,0,2019-08-01 00:00:00,26,4,0,-33.409089,-70.568177,T421 00I,...,9492.0,1,0,0,0,1,0,0,ZN-5344_0,0
1,182892,30011,4,2019-08-01 00:00:30,8,4,0,-33.408872,-70.567463,T421 00I,...,9493.0,1,0,0,0,1,0,0,ZN-5344_0,0
2,182893,30362,6,2019-08-01 00:01:00,58,4,0,-33.407944,-70.563855,T421 00I,...,9494.0,1,0,0,0,1,0,0,ZN-5344_0,0
3,182894,30787,1,2019-08-01 00:01:30,48,4,0,-33.407622,-70.559355,T421 00I,...,9495.0,1,0,0,0,1,0,0,ZN-5344_0,0
4,182924,31105,0,2019-08-01 00:02:00,0,4,4,-33.407989,-70.555974,T421 00I,...,23455.0,1,0,0,0,1,0,0,ZN-5344_0,0


time: 53.3 s


In [6]:
expeditions_count = gps.groupby('exp_id').size().reset_index(name='count')
exp_list = expeditions_count[expeditions_count["count"] > thresh]["exp_id"]
gps = gps[gps["exp_id"].isin(exp_list)]

time: 2.26 s


In [7]:
%load_ext Cython

time: 606 ms


In [8]:
%%cython
import cython
import numpy as np
cimport numpy as np

cpdef float [:, :] pair_gps(float [:, :] expedition, float delta, int quant, int jump=1):
    cdef np.ndarray[np.float32_t, ndim=2] output = -np.ones([expedition.shape[0]*quant, 2], dtype = np.float32)
    cdef int i, j
    cdef int l = 0
    cdef int from_init_col = 0
    cdef int to_rout_col = 1
    cdef float dif_dist1, dif_dist2
    cdef float last_inrout, last_torout
    for i in range(0, expedition.shape[0], jump):
        if l >= expedition.shape[0]*quant:
            break
        last_inrout = expedition[i, from_init_col]
        last_torout = expedition[i, to_rout_col]
        for j in range(i + 1, expedition.shape[0]):
            if l >= expedition.shape[0]*quant:
                break
            dif_dist1 = np.abs(last_inrout - expedition[j, from_init_col])
            dif_dist2 = np.abs(last_torout - expedition[j, to_rout_col])
            if dif_dist1 + dif_dist2 > delta:
                output[l, 0] = i
                output[l, 1] = j
                l += 1
                last_inrout = expedition[j, from_init_col]
                last_torout = expedition[j, to_rout_col]
    return output

time: 12 ms


In [9]:
def pair(expedition, delta, quant, jump):
    distances = expedition[["DistanciaInicio", "DistanciaRuta"]].to_numpy(dtype=np.float32)
    indexes = np.asarray(pair_gps(distances, delta, quant, jump))
    indexes = indexes[indexes[:, 0] >= 0]
    if indexes.shape[0] > 0:
        gps1 = expedition.iloc[indexes[:, 0], :].add_suffix('1').reset_index()
        gps2 = expedition.iloc[indexes[:, 1], :].add_suffix('2').reset_index()
        concatenated_gps = pd.concat([gps1, gps2], axis=1).drop(columns=["exp_id1"]).rename(columns={'exp_id2': 'exp_id', 'Servicio2': 'Servicio'})
        concatenated_gps["Diferencia"] = (concatenated_gps["GPS_time2"] - concatenated_gps["GPS_time1"]).dt.total_seconds()
        concatenated_gps = concatenated_gps.drop(columns=["index", "GPS_time1", "GPS_time2", "Servicio1"])
        return concatenated_gps
    else:
        return None

time: 2 ms


In [10]:
def pair_every_gps(gps, delta, quant, jump):
    servicios = gps["Servicio"].unique()
    output = []
    total = len(servicios)
    for idx, servicio in enumerate(servicios):
        print('Servicio {}/{}'.format(idx+1, total))
        gps_servicio = gps[gps["Servicio"] == servicio]
        expeditions = gps_servicio["exp_id"].unique()
        total_exp = len(expeditions)
        for j, expedition in enumerate(expeditions):
            print('\rExpedicion {}/{}'.format(j+1, total_exp), end='')
            gps_expedition = gps_servicio[gps_servicio["exp_id"] == expedition]
            pairs = pair(gps_expedition, delta, quant, jump)
            if not pairs is None:
                output.append(pairs)
        print('\n')
    if len(output) == 0:
        return None
    return pd.concat(output, axis=0, ignore_index=True)

time: 2 ms


In [11]:
def pair_gps_day(gps, delta, quant, jump, file_path, start=0):
    dates = gps["GPS_time"].map(pd.Timestamp.date).unique()
    total_dates = len(dates)
    for idx, d in enumerate(dates):
        if idx >= start:
            print("Fecha numero: {}/{}".format(idx+1, total_dates))
            gps_date = gps[gps["GPS_time"].dt.date == d]
            paired_gps = pair_every_gps(gps_date, thresh, n, jump)
            if paired_gps is not None:
                paired_gps = paired_gps.drop(columns=["Unnamed: 01", "Ind11", "Ind21", "Ind31", "Unnamed: 0.11", "Unnamed: 0.1.11", "Unnamed: 02", "Ind12", "Ind22", "Ind32", "Unnamed: 0.12", "Unnamed: 0.1.12", "dia_habil2", "dia_Monday2", "dia_Saturday2", "dia_Sunday2", "dia_Thursday2", "dia_Tuesday2", "dia_Wednesday2"])
                # Guardar base de datos en un archivo haciendo append
                header = idx == start
                paired_gps.to_csv(file_path, sep=',', index=False, mode='a+', header=header)
    print("Done :)")

time: 2.01 ms


# Separación de base de datos: train, val, test

In [12]:
testset = gps[gps["GPS_time"].dt.day.isin(test_days)]
restset = gps[~gps["GPS_time"].dt.day.isin(test_days)]

expeditions = gps["exp_id"].unique()

train_exp, val_exp = train_test_split(expeditions, test_size=train_val_perc)

trainset = restset[restset.exp_id.isin(train_exp)]
valset = restset[restset.exp_id.isin(val_exp)]

print("Cantidad de pulsos de entrenamiento: {}".format(trainset.shape[0]))
print("Cantidad de pulsos de validacion: {}".format(valset.shape[0]))
print("Cantidad de pulsos de prueba: {}".format(testset.shape[0]))

Cantidad de pulsos de entrenamiento: 8637495
Cantidad de pulsos de validacion: 2170693
Cantidad de pulsos de prueba: 1639434
time: 5.04 s


In [13]:
pair_gps_day(valset, thresh, n, jump, val_path)

Fecha numero: 1/27
Servicio 1/18
Expedicion 33/33

Servicio 2/18
Expedicion 35/35

Servicio 3/18
Expedicion 40/40

Servicio 4/18
Expedicion 31/31

Servicio 5/18
Expedicion 3/3

Servicio 6/18
Expedicion 3/3

Servicio 7/18
Expedicion 21/21

Servicio 8/18
Expedicion 22/22

Servicio 9/18
Expedicion 33/33

Servicio 10/18
Expedicion 2/2

Servicio 11/18
Expedicion 27/27

Servicio 12/18
Expedicion 6/6

Servicio 13/18
Expedicion 32/32

Servicio 14/18
Expedicion 2/2

Servicio 15/18
Expedicion 21/21

Servicio 16/18
Expedicion 38/38

Servicio 17/18
Expedicion 5/5

Servicio 18/18
Expedicion 29/29

Fecha numero: 2/27
Servicio 1/17
Expedicion 31/31

Servicio 2/17
Expedicion 42/42

Servicio 3/17
Expedicion 44/44

Servicio 4/17
Expedicion 43/43

Servicio 5/17
Expedicion 2/2

Servicio 6/17
Expedicion 6/6

Servicio 7/17
Expedicion 2/2

Servicio 8/17
Expedicion 21/21

Servicio 9/17
Expedicion 22/22

Servicio 10/17
Expedicion 43/43

Servicio 11/17
Expedicion 32/32

Servicio 12/17
Expedicion 9/9

Servicio 1

In [14]:
pair_gps_day(testset, thresh, n, jump, test_path)

Fecha numero: 1/4
Servicio 1/19
Expedicion 172/172

Servicio 2/19
Expedicion 209/209

Servicio 3/19
Expedicion 198/198

Servicio 4/19
Expedicion 195/195

Servicio 5/19
Expedicion 363/363

Servicio 6/19
Expedicion 204/204

Servicio 7/19
Expedicion 60/60

Servicio 8/19
Expedicion 191/191

Servicio 9/19
Expedicion 164/164

Servicio 10/19
Expedicion 141/141

Servicio 11/19
Expedicion 77/77

Servicio 12/19
Expedicion 157/157

Servicio 13/19
Expedicion 397/397

Servicio 14/19
Expedicion 204/204

Servicio 15/19
Expedicion 192/192

Servicio 16/19
Expedicion 197/197

Servicio 17/19
Expedicion 148/148

Servicio 18/19
Expedicion 131/131

Servicio 19/19
Expedicion 118/118

Fecha numero: 2/4
Servicio 1/19
Expedicion 140/140

Servicio 2/19
Expedicion 167/167

Servicio 3/19
Expedicion 168/168

Servicio 4/19
Expedicion 168/168

Servicio 5/19
Expedicion 5/5

Servicio 6/19
Expedicion 13/13

Servicio 7/19
Expedicion 5/5

Servicio 8/19
Expedicion 109/109

Servicio 9/19
Expedicion 102/102

Servicio 10/19
E

In [15]:
pair_gps_day(trainset, thresh, n, jump, train_path)

Fecha numero: 1/27
Servicio 1/19
Expedicion 142/142

Servicio 2/19
Expedicion 166/166

Servicio 3/19
Expedicion 157/157

Servicio 4/19
Expedicion 144/144

Servicio 5/19
Expedicion 288/288

Servicio 6/19
Expedicion 181/181

Servicio 7/19
Expedicion 52/52

Servicio 8/19
Expedicion 147/147

Servicio 9/19
Expedicion 961/961

Servicio 10/19
Expedicion 131/131

Servicio 11/19
Expedicion 72/72

Servicio 12/19
Expedicion 122/122

Servicio 13/19
Expedicion 308/308

Servicio 14/19
Expedicion 139/139

Servicio 15/19
Expedicion 164/164

Servicio 16/19
Expedicion 164/164

Servicio 17/19
Expedicion 143/143

Servicio 18/19
Expedicion 107/107

Servicio 19/19
Expedicion 127/127

Fecha numero: 2/27
Servicio 1/19
Expedicion 147/147

Servicio 2/19
Expedicion 170/170

Servicio 3/19
Expedicion 178/178

Servicio 4/19
Expedicion 150/150

Servicio 5/19
Expedicion 304/304

Servicio 6/19
Expedicion 172/172

Servicio 7/19
Expedicion 54/54

Servicio 8/19
Expedicion 144/144

Servicio 9/19
Expedicion 696/696

Servic

# Lectura y normalización de la base de datos

In [16]:
chunksize = 1e6
train = pd.read_csv(train_path, sep=',', header=0, chunksize=chunksize)
val = pd.read_csv(val_path, sep=',', header=0, chunksize=chunksize)

time: 151 ms


In [17]:
train = pd.read_csv(train_path, sep=',', header=0, chunksize=chunksize)
total_pulses = np.sum([chunk.shape[0] for chunk in train])

time: 2min 54s


In [18]:
columns_to_norm = ["DistanciaInicio1", "DistanciaRuta1", "Latitud1", "Longitud1", "DistanciaInicio2", "DistanciaRuta2", "Latitud2", "Longitud2"]

train = pd.read_csv(train_path, sep=',', header=0, chunksize=chunksize)
mean_train = [chunk[columns_to_norm].mean(axis=0) for chunk in train]
mean_train = pd.concat(mean_train, axis=1).mean(axis=1)
mean_train.to_csv(mean_path, sep=',')
mean_train

DistanciaInicio1     3666.503125
DistanciaRuta1         43.477459
Latitud1              -33.485385
Longitud1             -70.680034
DistanciaInicio2    17135.125172
DistanciaRuta2         52.926037
Latitud2              -33.473240
Longitud2             -70.678022
dtype: float64

time: 3min 2s


In [19]:
train = pd.read_csv(train_path, sep=',', header=0, chunksize=chunksize)
std_train = [chunk[columns_to_norm].std(axis=0) for chunk in train]
std_train = pd.concat(std_train, axis=1).mean(axis=1)
std_train.to_csv(std_path, sep=',')
std_train

DistanciaInicio1    4168.369521
DistanciaRuta1       312.197441
Latitud1               0.071244
Longitud1              0.068742
DistanciaInicio2    9185.529454
DistanciaRuta2       471.672063
Latitud2               0.060014
Longitud2              0.061156
dtype: float64

time: 3min 18s


In [20]:
# Entrenar modelo con DistanciaInicio1, DistanciaRuta1, Latitud1, Longitud1 => Servicio, ¿Podemos eliminar columna servicio?
# Entrenar modelo con Lat/Long y sin Lat/Long
# Eliminar exp_id para primera iteracion

Error in callback <function LineWatcher.stop at 0x00000206628C60D0> (for post_run_cell):


AssertionError: 